## Part 1

In [1]:
import pandas as pd

In [2]:
documents = []
_id = 1

# read in documents
with open('input.txt') as f:
    doc = {}
    for line in f:
        if not line.strip(): # empty line
            doc['_id'] = _id 
            if len(doc) > 0: documents.append(doc)
            doc = {}
            _id += 1
        else:
            chunks = line.split()
            for chunk in chunks:
                key, value = chunk.split(':')
                doc[key] = value
    doc['_id'] = _id 
    if len(doc) > 0: documents.append(doc)
                
print(documents[:3])
print(documents[-3:])
print(len(documents))

[{'eyr': '2024', 'pid': '662406624', 'hcl': '#cfa07d', 'byr': '1947', 'iyr': '2015', 'ecl': 'amb', 'hgt': '150cm', '_id': 1}, {'iyr': '2013', 'byr': '1997', 'hgt': '182cm', 'hcl': '#ceb3a1', 'eyr': '2027', 'ecl': 'gry', 'cid': '102', 'pid': '018128535', '_id': 2}, {'hgt': '61in', 'iyr': '2014', 'pid': '916315544', 'hcl': '#733820', 'ecl': 'oth', '_id': 3}]
[{'byr': '1987', 'hcl': '#7d3b0c', 'iyr': '2018', 'hgt': '180cm', 'ecl': 'blu', 'eyr': '2029', 'pid': '878348021', '_id': 280}, {'hgt': '183cm', 'cid': '98', 'byr': '1953', 'hcl': '#866857', 'eyr': '2021', 'iyr': '2012', 'pid': '158898193', '_id': 281}, {'eyr': '2030', 'pid': '039638764', 'ecl': 'hzl', 'hgt': '190cm', 'byr': '1926', 'cid': '294', 'hcl': '#b6652a', 'iyr': '2017', '_id': 282}]
282


In [3]:
table = pd.DataFrame(documents)
table_optional_cid = table.drop('cid', axis=1)
table_optional_cid[~table_optional_cid.isna().any(axis=1)]

,eyr,pid,hcl,byr,iyr,ecl,hgt,_id
0,2024,662406624,#cfa07d,1947,2015,amb,150cm,1
1,2027,018128535,#ceb3a1,1997,2013,gry,182cm,2
3,2026,726519569,#a97842,1980,2011,grn,184cm,4
4,2027,619743219,#6b5442,1980,2012,grn,176cm,5
5,2020,982796633,#602927,1969,2014,brn,164cm,6
...,...,...,...,...,...,...,...,...
276,2026,836559549,#c0946f,1981,2011,amb,167cm,277
277,2026,111085991,#602927,1920,2011,blu,182cm,278
278,2027,284436132,#6b5442,1929,2010,oth,75in,279
279,2029,878348021,#7d3b0c,1987,2018,blu,180cm,280


## Part 2

In [4]:
table = pd.DataFrame(documents)

table[['hgt', 'hgt_unit']] = table['hgt'].str.extract(r'(\d+)(\D+)', expand=True)

table = table.drop('cid', axis=1)
table['error_reason'] = ''
table.loc[table.isna().any(axis=1), 'error_reason'] += 'Null value '
table.loc[~table['byr'].str.fullmatch(r'[0-9]{4}', na=False), 'error_reason'] += 'byr format error '
table.loc[~table['iyr'].str.fullmatch(r'[0-9]{4}', na=False), 'error_reason'] += 'iyr format error '
table.loc[~table['eyr'].str.fullmatch(r'[0-9]{4}', na=False), 'error_reason'] += 'eyr format error '
table[['byr', 'iyr', 'eyr', 'hgt']] = table[['byr', 'iyr', 'eyr', 'hgt']].apply(pd.to_numeric)
table.loc[table['byr'] < 1920, 'error_reason'] += 'byr < 1920 '
table.loc[table['byr'] > 2002, 'error_reason'] += 'byr > 2002 '
table.loc[table['iyr'] < 2010, 'error_reason'] += 'iyr < 2010 '
table.loc[table['iyr'] > 2020, 'error_reason'] += 'iyr > 2020 '
table.loc[table['eyr'] < 2020, 'error_reason'] += 'eyr < 2020 '
table.loc[table['eyr'] > 2030, 'error_reason'] += 'eyr > 2030 '
table.loc[~table['hgt_unit'].isin(['cm', 'in']), 'error_reason'] += 'hgt unit wrong '
table.loc[(table['hgt_unit'] == 'cm') & (table['hgt'] < 150), 'error_reason'] += 'cm < 150 '
table.loc[(table['hgt_unit'] == 'cm') & (table['hgt'] > 193), 'error_reason'] += 'cm > 193 '
table.loc[(table['hgt_unit'] == 'in') & (table['hgt'] < 59), 'error_reason'] += 'in < 59 '
table.loc[(table['hgt_unit'] == 'in') & (table['hgt'] > 76), 'error_reason'] += 'in > 76 '
table.loc[~table['hcl'].str.fullmatch(r'\#[a-f0-9]{6}', na=False), 'error_reason'] += 'hcl format error '
table.loc[~table['ecl'].isin(['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth']), 'error_reason'] += 'ecl invalid '
table.loc[~table['pid'].str.fullmatch(r'[0-9]{9}', na=False), 'error_reason'] += 'pid format error '
table[table['error_reason'] == '']

,eyr,pid,hcl,byr,iyr,ecl,hgt,_id,hgt_unit,error_reason
0,2024.0,662406624,#cfa07d,1947.0,2015.0,amb,150.0,1,cm,
1,2027.0,018128535,#ceb3a1,1997.0,2013.0,gry,182.0,2,cm,
3,2026.0,726519569,#a97842,1980.0,2011.0,grn,184.0,4,cm,
4,2027.0,619743219,#6b5442,1980.0,2012.0,grn,176.0,5,cm,
5,2020.0,982796633,#602927,1969.0,2014.0,brn,164.0,6,cm,
...,...,...,...,...,...,...,...,...,...,...
276,2026.0,836559549,#c0946f,1981.0,2011.0,amb,167.0,277,cm,
277,2026.0,111085991,#602927,1920.0,2011.0,blu,182.0,278,cm,
278,2027.0,284436132,#6b5442,1929.0,2010.0,oth,75.0,279,in,
279,2029.0,878348021,#7d3b0c,1987.0,2018.0,blu,180.0,280,cm,
